In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from dotenv import load_dotenv
load_dotenv() 

import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [4]:
from src.inference import get_feature_store

In [20]:
feature_store = get_feature_store()

2025-03-04 14:03:36,813 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 14:03:36,843 INFO: Initializing external client
2025-03-04 14:03:36,844 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 14:03:37,302 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215642


In [19]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-04 14:02:51,234 INFO: Closing external client and cleaning up certificates.


Connection closed.
2025-03-04 14:02:51,264 INFO: Initializing external client
2025-03-04 14:02:51,265 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 14:02:52,075 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215642
Fetching data from 2025-02-03 19:02:51.223439+00:00 to 2025-03-04 18:02:51.223439+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.38s) 


In [6]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-03 19:00:00+00:00,2,0
1,2025-02-03 20:00:00+00:00,2,0
2,2025-02-03 21:00:00+00:00,2,0
3,2025-02-03 22:00:00+00:00,2,0
4,2025-02-03 23:00:00+00:00,2,0
...,...,...,...
174440,2025-03-04 13:00:00+00:00,263,100
174441,2025-03-04 14:00:00+00:00,263,146
174442,2025-03-04 15:00:00+00:00,263,126
174443,2025-03-04 16:00:00+00:00,263,113


In [7]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 179213
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174445 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174445 non-null  int32                  
 2   rides               174445 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [8]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [9]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 179213
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174445 non-null  datetime64[us]
 1   pickup_location_id  174445 non-null  int32         
 2   rides               174445 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [10]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [11]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,1,1,1,10,4,5,12,13,1,0,...,0,1,3,0,2,0,0,2,33,2025-02-25 03:00:00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-02-27 20:00:00
2,1,0,0,1,0,2,2,0,3,1,...,1,0,1,0,0,0,1,3,14,2025-02-20 00:00:00
3,0,0,2,5,2,0,0,2,0,5,...,0,0,0,0,1,2,3,1,39,2025-02-15 12:00:00
4,1,40,26,17,10,48,43,42,23,20,...,37,25,26,19,35,4,44,37,50,2025-02-21 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,127,16,6,126,113,152,86,73,4,29,...,146,15,82,125,149,22,143,15,107,2025-02-04 14:00:00
247,0,1,3,1,1,1,2,1,2,0,...,2,1,0,1,1,0,4,1,168,2025-02-28 05:00:00
248,2,1,7,2,8,3,1,1,1,1,...,0,0,1,0,1,3,2,1,97,2025-02-09 23:00:00
249,0,3,0,1,0,0,2,0,0,0,...,0,0,0,3,2,0,0,0,62,2025-02-26 13:00:00


In [12]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-04 13:59:09,740 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 13:59:09,768 INFO: Initializing external client
2025-03-04 13:59:09,768 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 13:59:11,704 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215642
Fetching data from 2025-02-03 18:59:09.740078+00:00 to 2025-03-04 17:59:09.740078+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.28s) 


In [13]:
current_date

Timestamp('2025-03-04 18:59:09.740078+0000', tz='Etc/UTC')

In [14]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-03 19:00:00
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,3,2025-03-03 19:00:00
2,1,3,3,6,2,3,6,2,0,1,...,5,1,2,2,3,0,1,1,4,2025-03-03 19:00:00
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-03 19:00:00
4,5,1,2,0,0,0,0,1,0,0,...,2,8,3,4,1,3,4,1,7,2025-03-03 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,259,2025-03-03 19:00:00
247,3,0,1,0,0,0,0,0,0,0,...,1,4,3,2,5,1,6,1,260,2025-03-03 19:00:00
248,24,17,16,11,3,1,3,0,0,2,...,16,25,36,32,26,33,43,45,261,2025-03-03 19:00:00
249,43,34,22,4,6,1,3,1,1,3,...,98,79,71,82,97,87,82,103,262,2025-03-03 19:00:00


In [15]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-04 13:59:32,224 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 13:59:32,231 INFO: Initializing external client
2025-03-04 13:59:32,231 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 13:59:32,724 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215642


In [16]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

python(23470) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [17]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,2.0
3,6,0.0
4,7,3.0
...,...,...
246,259,0.0
247,260,1.0
248,261,27.0
249,262,60.0


In [18]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 161, 138, 162, 237, 236, 230, 163, 142, 186], dtype=int32)